In [1]:
from wildcat.solver.qubo_solver import QuboSolver
from wildcat.network.local_endpoint import LocalEndpoint
from wildcat.annealer.simulated.simulated_annealer import SimulatedAnnealer
from wildcat.annealer.simulated.single_spin_flip_strategy import SingleSpinFlipStrategy
from wildcat.annealer.simulated.temperature_schedule import TemperatureSchedule
from wildcat.util.matrix import hamiltonian_energy
    
schedule = TemperatureSchedule(initial_temperature=1000, last_temperature=0.1, scale=0.99)
strategy = SingleSpinFlipStrategy(repetition=1000)
annealer = SimulatedAnnealer(schedule=schedule, strategy=strategy)
local_endpoint = LocalEndpoint(annealer=annealer)

#行列muを作成
mu = np.array([
    [0.0,0.2,0.3],
    [0.2,0.2,0.1],
    [0.1,0.3,0.3],
    [0.4,0.1,0.5],
    [0.5,0.3,0.1]
])

# T : 期間
T = mu.shape[0]
# N : 銘柄の数
N = mu.shape[1]
# K : 選ぶ銘柄の数
K = 10
# S : ２進数の桁数
S = 5
# M : 定数
M = 1

In [2]:
#クロネッカーデルタを定義
def delta(i,j):
    if(i == j):
        return 1
    else:
        return 0

In [3]:
# 行列Jを計算
J = np.empty((T,N,S,N,S),dtype = np.float64)
for t in range(T):
    for i in range(N):
        for a in range(S):
            for j in range(N):
                for b in range(S):
                    J[t,i,a,j,b] = -2**a * mu[t,i] * delta(i,j) * delta(a,b) + M * (2**(a + b) - 2**(1 + a) * K * delta(i,j) * delta(a,b))
J

array([[[[[ -19. ,    2. ,    4. ,    8. ,   16. ],
          [   1. ,    2. ,    4. ,    8. ,   16. ],
          [   1. ,    2. ,    4. ,    8. ,   16. ]],

         [[   2. ,  -36. ,    8. ,   16. ,   32. ],
          [   2. ,    4. ,    8. ,   16. ,   32. ],
          [   2. ,    4. ,    8. ,   16. ,   32. ]],

         [[   4. ,    8. ,  -64. ,   32. ,   64. ],
          [   4. ,    8. ,   16. ,   32. ,   64. ],
          [   4. ,    8. ,   16. ,   32. ,   64. ]],

         [[   8. ,   16. ,   32. ,  -96. ,  128. ],
          [   8. ,   16. ,   32. ,   64. ,  128. ],
          [   8. ,   16. ,   32. ,   64. ,  128. ]],

         [[  16. ,   32. ,   64. ,  128. ,  -64. ],
          [  16. ,   32. ,   64. ,  128. ,  256. ],
          [  16. ,   32. ,   64. ,  128. ,  256. ]]],


        [[[   1. ,    2. ,    4. ,    8. ,   16. ],
          [ -19.2,    2. ,    4. ,    8. ,   16. ],
          [   1. ,    2. ,    4. ,    8. ,   16. ]],

         [[   2. ,    4. ,    8. ,   16. ,   32. ]

In [4]:
#定数項の計算
C = T * M * K**2
C

500

In [5]:
#行列Qを計算
Q = np.zeros((T*N*S, T*N*S), dtype = np.float64)
for t in range(T):
    for i in range(N):
        for a in range(S):
            for j in range(N):
                for b in range(S):
                    Q[a + i*S + t*S*N,b + j*S + t*S*N] = J[t,i,a,j,b]
Q

array([[-19. ,   2. ,   4. , ...,   0. ,   0. ,   0. ],
       [  2. , -36. ,   8. , ...,   0. ,   0. ,   0. ],
       [  4. ,   8. , -64. , ...,   0. ,   0. ,   0. ],
       ...,
       [  0. ,   0. ,   0. , ..., -64.4,  32. ,  64. ],
       [  0. ,   0. ,   0. , ...,  32. , -96.8, 128. ],
       [  0. ,   0. ,   0. , ...,  64. , 128. , -65.6]])

In [6]:
#最適化計算の定義
solver = QuboSolver(Q)
def callback(arrangement):
    e = solver.hamiltonian_energy(arrangement)

In [7]:
#(-1,1)->(0,1)に変える関数
def chgangeExpression(vector):
    D = vector.shape[0]
    for x in range(D):
        if(vector[x] == -1):
            vector[x] = 0
    return vector

In [8]:
#エネルギーを計算する関数
def calcEnergy(vector, matrix, const):
    return vector.T.dot(matrix).dot(vector) + const

In [9]:
#繰り返し計算し、最小エネルギーを探す
MinEnergy = 0
MinArrangement = np.zeros(N*T, dtype = np.int)
for i in range(1000):
    arrangement = chgangeExpression(solver.solve(callback, endpoint=local_endpoint).result())
    if(i == 0):
        MinEnergy = calcEnergy(arrangement, Q, C)
        MinArrangement = arrangement
    else:
        if(calcEnergy(arrangement, Q, C) < MinEnergy):
            MinEnergy = calcEnergy(arrangement, Q, C)
            MinArrangement = arrangement

In [10]:
print("Energy: ", calcEnergy(MinArrangement, Q, C))
print("Spins: ", MinArrangement)

Energy:  -12.399999999999977
Spins:  [0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0
 0]


In [11]:
result = np.zeros((N, T), dtype = np.int)
i = 0
for t in range(T):
    for i in range(N):
        for a in range(S):
            result[i, t] += 2**a * MinArrangement[a + i*S + t*S*N]
print("Result: \n", result)

Result: 
 [[0 2 8 4 5]
 [7 3 2 1 0]
 [3 5 0 5 5]]
